In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
# from scipy import stats

from utils import (
    detrend, monthly_anomalies,reg_slope,
    standardize, corr_with_ttest
)

In [2]:
eq = dict(lat=0,lon=slice(140,260))

time = slice('1993-01','2019-11')

In [3]:
t_grid_file = xr.open_zarr('data/ORCA025.L46-KFS006_TroPA.zarr')

In [4]:
def nlti(ssh,d20):
    slope = reg_slope(d20, ssh, dim="time")

    sshmd20 = (
        ssh - slope * d20
    ).rename("NLTI")

    nlti=detrend(
        sshmd20
    )
    return nlti

### 1. Prepare data
#### 1.1 SSH 

In [5]:
ssh_o = t_grid_file.sossheig.sel(**eq)
ssh_o.attrs["units"] = "m"
ssh_ab = monthly_anomalies(ssh_o.sel(time=time))

#### 1.2 D20 

In [6]:
d20_o = -1 * t_grid_file.d20.sel(**eq)

In [7]:
d20_a = monthly_anomalies(d20_o.sel(time=time))

#### 1.3 linear data (NEMO fitting) 

In [8]:
eta = (
    xr.open_zarr("data/lmmm_eta_nemo.zarr/")
    .eta
    .interp(lat=0)
    .sel(lon=slice(140,260))
    .sel(mode=slice(1, 3))
)

In [9]:
eta_b = eta.assign_coords(
    {'time':(eta.time-np.timedelta64(1314900,'s')).data}
).interp(time=ssh_ab.time).fillna(0.)

#### 1.4 fitting (to NEMO)

In [10]:
def fitting(ssha,eta_x):
    mask = ~(
        np.isnan(ssha).all('time')
        | np.isnan(eta_x.sel(mode=1)).all('time')
    )
    a = np.vstack(
        [
            eta_x.sel(mode=k)[:, mask].values.flatten()
            for k in (1,2,3)
        ]
    ).T
    a_inv = np.linalg.pinv(a)

    b = ssha[:, mask].values.flatten()

    coeff = a_inv.dot(b)
    coeff = xr.DataArray(
        coeff,
        coords=(eta_x.sel(mode=slice(1, 3)).mode,),
        dims=("mode",)
    )
    return coeff

In [11]:
coeff_b = fitting(ssh_ab, eta_b)

In [12]:
sshli_b = (coeff_b * eta_b).sum("mode").compute()

#### 1.5 Fitting to AVISO 

In [13]:
eta_aviso = (
    xr.open_zarr("data/lmmm_eta_woa13.zarr/")
    .eta
    .interp(lat=0)
    .sel(lon=slice(140,260))
    .sel(mode=slice(1, 3))
)

In [14]:
sla = xr.open_dataset(
    'data/aviso199301_202002.nc'
).sla.interp(lat=0., lon=eta.lon).fillna(0.)

In [15]:
ssh_c = monthly_anomalies(sla.sel(time=time))

In [16]:
eta_c = eta_aviso.assign_coords(
    {'time':(eta.time-np.timedelta64(1314900,'s')).data}
).interp(time=ssh_ab.time).fillna(0.)

In [17]:
coeff_c = fitting(ssh_c, eta_c)

In [18]:
sshli_c = (coeff_c * eta_c).sum("mode")

### 3. Fig7: Create NLTIs

In [19]:
ssh_ab = detrend(ssh_ab.compute())
d20_a = detrend(d20_a.compute())
d20_b = -1*detrend(sshli_b.compute())
ssh_c = detrend(ssh_c.compute())
d20_c = -1*detrend(sshli_c.compute())

In [20]:
nlti_nemo = nlti(ssh_ab,d20_a)
nlti_nemolin = nlti(ssh_ab,d20_b)
nlti_aviso = nlti(ssh_c,d20_c)

## Write output

In [21]:
!rm -rf data_fig7.zarr
xr.Dataset(
    dict(
        nlti_nemo=nlti_nemo,
        nlti_nemolin=nlti_nemolin,
        nlti_aviso=nlti_aviso
    )
).to_zarr("data_fig7.zarr");